# Классификация текстов: спам-фильтр для SMS

Необходимо построить классификатор текстов по данным открытого датасета с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham").

**1. Загрузка [данных](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/).**

In [1]:
folder_path = r'C:\Users\Xiaomi\Desktop\Data Science\5. Прикладные задачи анализа данных'
file_name = r'SMSSpamCollection.txt'
file_path = folder_path + '\\' + file_name

In [2]:
with open(file_path, 'r', encoding='charmap') as new_file:
    data = new_file.read()

In [3]:
data = data.split('\n')

In [4]:
# Удаление пустого элемента
del data[-1]

In [5]:
print('data:', len(data))

data: 5574


**2. Подготовка списков для дальнейшей работы.**

1 - спам, 0 - "не спам"

In [6]:
messages = list()
labels = list()

for message in data:
    
    text = message.split('\t')
    
    messages.append(text[1])
    
    if text[0] == 'ham':
        labels.append(0)
        
    elif text[0] == 'spam':
        labels.append(1)

In [7]:
print('messages:', len(messages))
print('labels', len(labels))

messages: 5574
labels 5574


In [8]:
import numpy as np

np.unique(ar=labels, return_counts=True)

(array([0, 1]), array([4827,  747], dtype=int64))

**3. Подготовка матрицы признаков.**

In [58]:
# модули для построения признаков на текстах
from sklearn.feature_extraction.text import CountVectorizer

In [210]:
vectorizer = CountVectorizer()

In [211]:
X = vectorizer.fit_transform(messages)

**4. Оценка качества классификации текстов при использовании логистической регрессии.**

In [11]:
# линейные классификаторы
from sklearn.linear_model import LogisticRegression

# модуль кросс-валидации
from sklearn.model_selection import cross_val_score

In [95]:
cross_val_score(LogisticRegression(solver='lbfgs', random_state=2), X, labels, scoring='f1', cv=10).mean()

0.9318851880515823

**5. Обучение классификатора.**

In [96]:
classifier = LogisticRegression(solver='lbfgs', random_state=2).fit(X, labels)

**6. Проверка работы классификатора на примерах сообщений (1,1,0,0,0).**

0 - не спам, 1 - спам

In [103]:
test_messges = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"
]

In [185]:
X_test = vectorizer.transform(test_messges)
print(classifier.predict(X_test))

[1 1 0 0 0]


**7. Сравнение качества классификации при добавлении в признаки n-граммы для разных диапазонов n - только биграмм, только триграмм, всего вместе - униграммы, биграммы и триграммы.**

In [186]:
# модуль пайплайн
from sklearn.pipeline import Pipeline

In [191]:
# Возвращает пайплайн для текстовой классификации
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [200]:
for param in [(2,2), (3,3), (1,3)]:
    print('ngram_range',param)
    print(cross_val_score(text_classifier(CountVectorizer(ngram_range=param), LogisticRegression(solver='lbfgs', random_state=2)), messages, labels, scoring='f1', cv=10).mean())
    print()

ngram_range (2, 2)
0.816782323945987

ngram_range (3, 3)
0.7250161555467377

ngram_range (1, 3)
0.9236977114588596



Статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

**8. Сравнение качества классификации при добавлении в признаки n-граммы для разных диапазонов n - только биграмм, только триграмм, всего вместе - униграммы, биграммы и триграммы при использовании вместо логистической регрессии наивного Байеса.**

In [202]:
from sklearn.naive_bayes import MultinomialNB

In [207]:
print('Не верно!!!')

for param in [(2,2), (3,3), (1,3)]:
    print('ngram_range',param)
    print(cross_val_score(text_classifier(CountVectorizer(ngram_range=param), MultinomialNB()), messages, labels, scoring='f1', cv=10).mean())
    print()

ngram_range (2, 2)
0.9323117762969109

ngram_range (3, 3)
0.871265305963816

ngram_range (1, 3)
0.9479591028904112



По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой.

In [218]:
for param in [(2,2), (3,3), (1,3)]:
    print('ngram_range',param)
    vectorizer = CountVectorizer(ngram_range=param)
    X = vectorizer.fit_transform(messages)
    classifier = MultinomialNB().fit(X, labels)
    print(cross_val_score(classifier, X, labels, scoring='f1', cv=10).mean())
    print()

ngram_range (2, 2)
0.6434846004951214

ngram_range (3, 3)
0.37758088051034894

ngram_range (1, 3)
0.8863675289084053



Наивный Байес значительно сильнее линейного классификатора страдает от нехватки статистики по биграммам и триграммам.

**9. Используем в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах.**

In [219]:
# модули для построения признаков на текстах
from sklearn.feature_extraction.text import TfidfVectorizer

In [221]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(messages)
cross_val_score(LogisticRegression(solver='lbfgs', random_state=2), X, labels, scoring='f1', cv=10).mean()

0.8519515251846876